In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

def show_mid_slices(image):
    """ Function to display row of image middle slices """
    shape = image.shape
    slices = [image[int(shape[0]/2), :, :],
             image[:, int(shape[1]/2), :],
             image[:, :, int(shape[2]/2)]]
    
    fig, axes = plt.subplots(1, len(slices), figsize=(500,200))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

        
def show_vol_slices(image):
    """ Function to display slices from several volumes """
    shape = image.shape
    vols = shape[3]
    slices = image[:, :, int(shape[2]/2), :]
    
    fig, axes = plt.subplots(int(vols/5)+1, 5, figsize=(50,20*(int(vols/5)+1)))
    print((500,200*(int(vols/5)+1)))
    for i in range(vols):
        if vols > 5:
            axes[int(i/5),i%5].imshow(slices[:,:,i].T, cmap="gray", origin="lower")
        else:
            axes[i].imshow(slices[:,:,i].T, cmap="gray", origin="lower")

In [ ]:
# pa and ap parameters are mandatory
pa = ""
ap = ""
# bvec and bval need to be specified only for the main encoding direction
# and only if the basename is different from the nifti file
bvec = ""
bval = ""
# the main encoding direction (PA or AP)
main_dir = "PA"
# the readout may be omitted if it is the same for both directions
readout_pa = 0.1
readout_ap = 0.1
# the position of the actual b0 volumes
pa_b0 = 0
ap_b0 = 0

In [ ]:
if main_dir == "PA":
    data = pa
    ix = 1
else:
    data = ap
    ix = 2
    
if bvec == "":
    bvec = '/'.join([*data.split('/')[:-1], '']) + data.split('/')[-1].split('.')[0] + '.bvec'
    bval = '/'.join([*data.split('/')[:-1], '']) + data.split('/')[-1].split('.')[0] + '.bval'

# Sample slices for PA image

In [ ]:
pa_img = nib.load(pa)
pa_img_data = pa_img.get_fdata()

show_mid_slices(pa_img_data[:,:,:,pa_b0])  

# Sample slices for AP image

In [ ]:
ap_img = nib.load(ap)
ap_img_data = ap_img.get_fdata()

show_mid_slices(ap_img_data[:,:,:,ap_b0])

In [ ]:
%%bash -s "$pa" "$ap"
dwidenoise $1 PA_denoised.nii
mrdegibbs PA_denoised.nii PA_unringed.nii
dwidenoise $2 AP_denoised.nii
mrdegibbs AP_denoised.nii AP_unringed.nii

# Sample slices for unringed PA image

In [ ]:
pa_img = nib.load('PA_unringed.nii')
pa_img_data = pa_img.get_fdata()

show_mid_slices(pa_img_data[:,:,:,pa_b0]) 

# Sample slices for unringed AP image

In [ ]:
ap_img = nib.load('AP_unringed.nii')
ap_img_data = ap_img.get_fdata()

show_mid_slices(ap_img_data[:,:,:,ap_b0])

In [ ]:
%%bash -s "$pa_b0" "$ap_b0" "$readout_pa" "$readout_ap"
fslroi PA_unringed.nii b0_blip_up.nii.gz $1 1
fslroi AP_unringed.nii b0_blip_down.nii.gz $2 1
fslmerge -t b0_blip_up_down.nii.gz b0_blip_up.nii.gz b0_blip_down.nii.gz

printf "0 1 0 $3\n0 -1 0 $4" > params.txt

topup --imain=b0_blip_up_down --datain=params.txt --config=b02b0.cnf --out=topup_results --iout=hifi

fslmaths hifi -Tmean hifi
bet hifi hifi_brain -m

# Topup results

In [ ]:
hifi_img = nib.load('hifi.nii.gz')
hifi_img_data = hifi_img.get_fdata()

show_mid_slices(hifi_img_data)

In [ ]:
%%bash -s "$data" "$ix" "$bvec" "$bval"
vols=$(mrinfo $1 | grep "Dimensions" | cut -d 'x' -f 4 | tr -d ' ')
indx=""
for  ((i=1; i<=$vols; i+=1)); do indx="$indx $2"; done
echo $indx > index.txt

eddy --imain=$1 --mask=hifi_brain_mask --acqp=params.txt --index=index.txt \
    --bvecs=$3 --bvals=$4 --topup=topup_results --out=eddy_corrected --data_is_shelled

# Eddy results

In [ ]:
eddy_img = nib.load('eddy_corrected.nii.gz')
eddy_img_data =eddy_img.get_fdata()

show_vol_slices(eddy_img_data)